# Spatial Join using Geopandas - MLB Ballparks example

In this example, we're going to demonstrate how to conduct a spatial join ("sjoin") using two shapefiles. We will be working with two datasets. The first dataset is a shapefile of Major League Baseball ("MLB") ballparks with their geospatial coordinates (longitude, latitude). The MLB ballparks can be found at this link: https://www.sciencebase.gov/catalog/item/4f4e4a48e4b07f02db6230c7. 

The other is a shapefile of cities from the U.S. Census Bureau. The shapefile is available through Census Bureau's TIGER/Line shapefile archive. The shapefile used in this example can be found here: https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2018&layergroup=Urban+Areas. 

The goal is to match the MLB ballparks with the cities and states that they're located in. The sjoin function helps us achieve this goal.

## Importing libraries

In [5]:
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon
#from shapely.geometry.polygon import Polygon
from pandas import Series, DataFrame
import geopandas as gpd

## First, we read in the Census Bureau shapefile

In [42]:
df_cities = gpd.read_file("cb_2018_us_ua10_500k.shp")

We use the geopandas function "read_file" to read in the shapefiles. This function transforms the data in the shapefile as a GeoDataframe, which is similiar to a Pandas DataFrame but with a geometry column that has the shape (point, line, polygon, multipolygon, etc.) with the corresponding spatial coordinates (longitude, latitude). We'll call our Census Bureau GeoDataFrame "df_cities".

In [43]:
df_cities.head()

,UACE10,AFFGEOID10,GEOID10,NAME10,LSAD10,UATYP10,ALAND10,AWATER10,geometry
0,88732,400C100US88732,88732,"Tucson, AZ",75,U,915276150,2078695,"MULTIPOLYGON (((-110.81345 32.11910, -110.7987..."
1,01819,400C100US01819,01819,"Alturas, CA",76,C,4933312,16517,"MULTIPOLYGON (((-120.54610 41.51264, -120.5459..."
2,22366,400C100US22366,22366,"Davenport, IA--IL",75,U,357345121,21444164,"MULTIPOLYGON (((-90.36678 41.53636, -90.36462 ..."
3,93322,400C100US93322,93322,"Waynesboro, PA--MD",76,C,45455957,88872,"MULTIPOLYGON (((-77.50746 39.71577, -77.50605 ..."
4,02548,400C100US02548,02548,"Angola, IN",76,C,23646957,3913803,"MULTIPOLYGON (((-85.01157 41.59300, -85.00589 ..."


In df_cities, we have the geometry of our cities (or urban areas) which are multipolygons that have several geospatial coordinates that represent the boundaries of the cities.

## Next, we read in the MLB Ballparks shapefile

In [16]:
df_ballparks = gpd.read_file("Stadiums_MLB.shp")

We'll name this GeoDataFrame "df_ballparks". Let's see what's included in df_ballparks.

In [19]:
df_ballparks.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 31 columns):
SECTOR        30 non-null object
SUBSECTOR     30 non-null object
PRIMARY_TY    30 non-null object
SEC_CLASS     30 non-null object
DATE_CREAT    30 non-null object
COMP_AFFIL    0 non-null object
NAME1         30 non-null object
NAME2         0 non-null object
NAME3         0 non-null object
ADDRESS1      30 non-null object
ADDRESS2      0 non-null object
PO_BOX        0 non-null object
PO_ZIP        0 non-null object
CITY          30 non-null object
STATE         30 non-null object
ZIP           30 non-null float64
ZIP_4         10 non-null object
COUNTY        30 non-null object
HSIP_AOI      30 non-null object
FEMA_REGIO    30 non-null float64
LATITUDE      30 non-null float64
LONGITUDE     30 non-null float64
RELIABILIT    30 non-null object
COORSOURCE    30 non-null object
COMMENTS_1    14 non-null object
LEAGUE        30 non-null object
DIVISION      30 non-null object


We'll subset the df_ballparks by just keeping only the following columns:
 - NAME1
 - CITY
 - STATE
 - TEAM
 - geometry

In [45]:
df_ballparks = df_ballparks[['NAME1','CITY','STATE','TEAM','geometry']]
df_ballparks.head()

,NAME1,CITY,STATE,TEAM,geometry
0,Angel Stadium of Anaheim,Anaheim,CA,Los Angeles Angels of Anaheim,POINT (-117.88204 33.80129)
1,Chase Field,Phoenix,AZ,Arizona Diamondbacks,POINT (-112.06673 33.44501)
2,Citizens Bank Park,Philadelphia,PA,Philadelphia Phillies,POINT (-75.16665 39.90618)
3,Comerica Park,Detroit,MI,Detroit Tigers,POINT (-83.04886 42.33912)
4,Coors Field,Denver,CO,Colorado Rockies,POINT (-104.99442 39.75664)


The geometry of the MLB ballparks are points, which is a simple geometry that has only one longitude and latitude coordinate.

## Now, we can do the spatial join (or sjoin)

We're going to do a right (how=right) sjoin (can also do a left join) using df_ballparks as the left GeoDataFrame and df_cities as the right GeoDataFrame. The operator keyword we're using is "within" (op="within") because we want to place the points of the MLB ballparks within the polygons of the cities. This will give us the city and state these MLB ballparks are located. We'll call our joined GeoDataFrame "join_df".

In [52]:
#https://stackoverflow.com/questions/58513123/link-each-point-in-one-geopandas-dataframe-to-polygons-in-another-dataframe
join_df=gpd.sjoin(df_ballparks, df_cities, how='right',op="within")

/Users/aaronwoodward/opt/anaconda3/lib/python3.7/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!({'init': 'epsg:4326'} != {'init': 'epsg:4269'})
  "(%s != %s)" % (left_df.crs, right_df.crs)


In [56]:
join_df.head(32)

,index_left,NAME1,CITY,STATE,TEAM,UACE10,AFFGEOID10,GEOID10,NAME10,LSAD10,UATYP10,ALAND10,AWATER10,geometry
index_right,,,,,,,,,,,,,,
702,0.0,Angel Stadium of Anaheim,Anaheim,CA,Los Angeles Angels of Anaheim,51445,400C100US51445,51445,"Los Angeles--Long Beach--Anaheim, CA",75,U,4500807839,57052601,"MULTIPOLYGON (((-117.50582 34.15048, -117.5014..."
702,5.0,Dodger Stadium,Los Angeles,CA,Los Angeles Dodgers,51445,400C100US51445,51445,"Los Angeles--Long Beach--Anaheim, CA",75,U,4500807839,57052601,"MULTIPOLYGON (((-117.50582 34.15048, -117.5014..."
119,1.0,Chase Field,Phoenix,AZ,Arizona Diamondbacks,69184,400C100US69184,69184,"Phoenix--Mesa, AZ",75,U,2968685079,12697349,"MULTIPOLYGON (((-111.43253 33.37751, -111.4321..."
1633,2.0,Citizens Bank Park,Philadelphia,PA,Philadelphia Phillies,69076,400C100US69076,69076,"Philadelphia, PA--NJ--DE--MD",75,U,5132240328,127543480,"MULTIPOLYGON (((-74.68335 39.98373, -74.67821 ..."
846,3.0,Comerica Park,Detroit,MI,Detroit Tigers,23824,400C100US23824,23824,"Detroit, MI",75,U,3460893786,94001027,"MULTIPOLYGON (((-82.76862 42.72953, -82.76760 ..."
3525,4.0,Coors Field,Denver,CO,Colorado Rockies,23527,400C100US23527,23527,"Denver--Aurora, CO",75,U,1726058510,38089345,"MULTIPOLYGON (((-104.71571 39.52160, -104.7154..."
3302,6.0,Fenway Park,Boston,MA,Boston Red Sox,09271,400C100US09271,09271,"Boston, MA--NH--RI",75,U,4853336622,201355196,"MULTIPOLYGON (((-70.75589 42.08023, -70.75351 ..."
1433,7.0,Great American Ball Park,Cincinnati,OH,Cincinnati Reds,16885,400C100US16885,16885,"Cincinnati, OH--KY--IN",75,U,2040396726,24339712,"MULTIPOLYGON (((-84.05054 39.05261, -84.05048 ..."
59,8.0,Progressive Field,Cleveland,OH,Cleveland Indians,17668,400C100US17668,17668,"Cleveland, OH",75,U,2004098980,11901959,"MULTIPOLYGON (((-81.09328 41.76613, -81.09313 ..."


We've successfully spatial joined the two GeoDataFrames. The joined GeoDataFrame, join_df, lists the items that were joined first. Notice, that there are two new columns as result of the sjoin: index_right and index_left. Index_right are the index references of the df_cities, and index_left are the index references of df_ballparks. After the joined items, there are a list of cities that do not have an MLB ballpark, therefore an sjoin was not possible to be performed. We can tidy join_df by removing these excess rows by simply selecting one of the MLB ballpark columns ("NAME1", "CITY", "STATE", "TEAM") and making it not equal (!=) to "NaN" (Not a number).

In [ ]:
join_df != join_df[join_df["NAME1"]!="NaN"]

Or, you can simply use the pandas ".dropna()" function to also achieve this.

In [ ]:
join_df = join_df.dropna()

Now, we have a cleaned joined GeoDataframe! There was one MLB ballpark that we were not able to join because the MLB team that plays its home games in a MLB ballpark outside the U.S. Can you identify the MLB team?

## Bonus: Converting geospatial coordinates into a geography

### Make a dataset of some famous landmarks and obtaining their geospatial coordinates

In [60]:
landmarks_df = pd.read_csv('Landmarks.csv')
landmarks_df.head()

,Landmark,City,Country,Lat,Lon
0,Empire State Building,New York City,USA,40.75,-73.99
1,Golden Gate Bridge,San Francisco,USA,37.82,-122.47
2,Washington Monument,Washington DC,USA,38.89,-77.03
3,House of Parliament,London,UK,51.50,-0.12
4,Eiffel Tower,Paris,France,48.86,2.29


We'll convert the DataFrame into a GeoDataFrame by transforming the longitude and latitude into a geometry.

In [61]:
landmarks_gdf = gpd.GeoDataFrame(landmarks_df, geometry=gpd.points_from_xy(landmarks_df.Lat, landmarks_df.Lon))
landmarks_gdf

,Landmark,City,Country,Lat,Lon,geometry
0,Empire State Building,New York City,USA,40.75,-73.99,POINT (40.750 -73.990)
1,Golden Gate Bridge,San Francisco,USA,37.82,-122.47,POINT (37.820 -122.470)
2,Washington Monument,Washington DC,USA,38.89,-77.03,POINT (38.890 -77.030)
3,House of Parliament,London,UK,51.50,-0.12,POINT (51.500 -0.120)
4,Eiffel Tower,Paris,France,48.86,2.29,POINT (48.860 2.290)
5,Colosseum,Rome,Italy,41.90,12.49,POINT (41.900 12.490)
6,Sydney Opera House,Sydney,Australia,-33.86,151.22,POINT (-33.860 151.220)
7,Tokyo Tower,Tokyo,Japan,35.66,139.75,POINT (35.660 139.750)


One important note to make. When working with shapefiles, the longitude coordinates are listed first in the geometry. In this example, latitude is listed first, which should be avoided because when plotting coordinates the first coordinate reference will be recognized as longitude. Therefore it's advised that longitude should be listed first, then latitude.

In [ ]:
                                ### END OF CODE ###